# Capítulo 9 - Acessando bancos de dados Entrez do NCBI

Entrez (https://www.ncbi.nlm.nih.gov/Web/Search/entrezfs.html) é um sistema de recuperação de dados que fornece aos usuários acesso aos bancos de dados do NCBI, tais como PubMed, GenBank, GEO e muitos outros. Você pode acessar o Entrez a partir da internet de modo manual ou usar o módulo `Bio.Entrez` do Biopython. Usando o Entrez como script, você pode, por exemplo, fazer pesquisas no PubMed ou baixar registros do GenBank.

O módulo `Bio.Entrez` faz uso do *Entrez Programming Utilities* (também conhecido como EUtils), consitindo em oito ferramentas que são descritas em detalhes na página do NCBI: https://www.ncbi.nlm.nih.gov/books/NBK25501/. Cada uma dessas ferramentas corresponde a uma função do módulo `Bio.Entrez` no Python, como descrito nas seções desse capítulo. Esse módulo grante que a URL correta seja usada nas pesquisas, e que as diretrizes responsáveis pelo acesso aos dados do NCBI sejam seguidas.

A saída retornada pelo EUtils geralamente é em um arquivo XML. Para analisar tal saída, você tem algumas opções:

1. Usar o analisador do `Bio.Entrez` para analisar a saída XML em um objeto Python;

2. Usar um dos analisadores XML disponível na biblioteca padrão do Python ou;

3. Ler a saída XML como texto bruto e analisá-lo por pesquisa e manipulação de string.

Veja a documentação do Python para os analisadores de XML em suas bibliotecas nativas (https://docs.python.org/pt-br/3.10/library/xml.html). Discutiremos aqui o analisador presente no Biopython com o módulo `Bio.Entrez`. Esse analisador pode ser usado para analisar os dados fornecidos por do Entrez por meio da programação, mas pode ser usado também para analisar dados em XML do NCBI Entrez que estão armazenados em um arquivo. Nesse último caso, o arquivo XML dele ser aberto em modo binário (e.g. **open('meu_arquivo.xml', 'rb')**) para que o analisador do XML no `Bio.Entrez` trabalhe corretamente.

O NCBI usa arquivos DTD (em inglês, *Document Type Definition*) para descrever a estrutura da informação contida em arquivos XML. Muitos dos arquivos DTD usados pelo NCBI estão incluídos no Biopython. O analisador `Bio.Entrez` faz uso dos arquivos DTD quando analisa um arquivo XML retornado pelo NCBI Entrez.

Ocasionalmente, você pode descobrir que o arquivo DTD associado com um arquivo XML específico está faltando no Biopython. Em particlar, isso acontece quando o NCBI atualiza os arquivos DTD. Se isso acontecer, **Entrez.read** mostrará uma mensagem de aviso com o nome e a URL do arquivo DTD faltante. O analisador procederá para o acesso do arquivo DTD faltante via internet, permitindo a análise do arquivo XML. Porém, o analisador é muito mais rápido se o arquivo DTD estiver disponível localmente. Para isso, baixe o arquivo DTD da URL mostrada na mensagem de aviso e coloque no diretório **...site-packages/Bio/Entrez/DTDs**, onde contém os outros arquivos DTD. Se você não tiver acesso a esse diretório, você pode colocar o arquivo DTD no diretório **~/.biopython/Bio/Entrez/DTDs**, onde o **~** representa o seu diretório principal. Uma vez que esse diretório é lido antes do primeiro, você também pode colocar versões recentes dos arquivos DTD se os presentes no primeiro diretório estiverem desatualizados. De modo alternativo, se você instalou o Biopython da fonte, pode adicionar o arquivo DTD ao diretório **Bio/Entrez/DTDs** do código fonte e reinstalar o Biopython. Isso instalará o novo arquivo DTD no local correto junto com os outros arquivos DTD.

O EUtils também pode gerar saídas em outros formatos, tais como arquivos FASTA ou GenBank para banco de dados de sequência, ou formato MedLine para o banco de dados de literatura; discutido na **Seção 9.13**.

As funções presentes no `Bio.Entrez` para acesso via programação ao Entrez também retorna os dados em formato binário ou em texto, dependendo do tipo de requisição do dado. Em muitos casos, essas funções retornam o dado em formato de texto, decodificando o dado obtido de NCBI Entrez para strings Python, assumindo que a codificação seja a UTF-8. Todavia, dados em XML são retornados no formato binário. A razão para isso é que a codificação é especificada no próprio documento XML, o que indica que você não saberá o tipo correto de codificação até que comece a análise do arquivo. Portanto, o analisador do `Bio.Entrez` aceita os dados em formato binário, extrai a codificação do XML e usa isso pra decodificar todo o texto presente no documento XML para strings Python, garantindo que todo o texto (em particular, idiomas além do Inglês) seja interpretado corretamente. Essa também é a razão do porque você deve abrir um arquivo XML no modo binário quando você quiser usar o analisador do `Bio.Entrez`.

## 9.1 Diretrizes do Entrez

Antes de usarmos o Biopython para acessar os recursos online do NCBI, vamos ver o que temos no Requisitos do usuário Entrez do NCBI (https://www.ncbi.nlm.nih.gov/books/NBK25497/). Se o NCBI achar que você está abusando dos seus sistemas, ele poderá e banirá o seu acesso!

Parafraseando:

- Para qualquer série com mais de 100 requisiões, faça isso nos finais de semana ou fora dos horários de pico dos EUA. Depende de você obedecer;

- Use o endereço web https://eutils.ncbi.nlm.nih.gov (cai na página do livro sobre o Entrez), não o endereço online do NCBI. O Biopython usa esse endereço web;

- Se você estiver usando uma chave API, você pode fazer até 10 consultas por segundo, caso contrário são no máximo 3 consultas por segundo. Isso é aplicado automaticamente pelo Biopython. Inclua o argumento **api_key='MinhaChaveAPI'** ou configure como uma variável do módulo:

In [1]:
from Bio import Entrez
Entrez.api_key = 'MinhaChaveAPI'

- Use o parâmetro opcional de email para que o NCBI entre em contato com você caso surja algum problema. Você também pode definir explicitamente como um parâmetro com cada chamada para Entrez, ou você pode configurar um email global:

In [2]:
from Bio import Entrez
Entrez.email = 'teste@example.com'

- `Bio.Entrez` então usará esse endereço de email com cada chamada do Entrez. O domímio **example.com** é um domínio de endereço reservado especialmente para documentação (RFC 2606). Por favor, **NÃO USE** um email aleatório, é melhor não informar um email nesse caso. Em caso de uso excessivo, o NCBI tentará contatar o usuário do email fornecido antes de bloquear o acesso às utilidades do Enrtez;

- Se você estiver usando o Biopython dentro de um pacote de software mario, use o parâmetro da ferramenta para especificar isso. Você pode definir explicitamente o nome da ferramenta como um parâmetro com cada chamada do Entrez, ou configurar o nome da ferramenta de modo global:

In [3]:
from Bio import Entrez
Entrez.tool = 'MeuScriptLocal'

- O parâmetro da ferramenta será o padrão Biopython e;

- Para grandes consultas o NCBI também recomenda o uso de seu recurso de histórico de seção (a string do cookie da seção WebEnv, veja a **Seção 9.16**). Isso é apenas um pouco mais complicado, mas não impossível.

Para concluir, seja sensato com seus níveis de uso. Se você pretende baixar muitos dados, considere outras opções. Por exemplo, se você quer acessar facilmente os genes humanos, considere buscar cada cromossomo pelo FTP como um arquivo GenBank e importe os importe para o seu banco de dados BioSQL (veja a **Seção 20.5** no Capítulo 20: ***link do capítulo 20***).

## 9.2 EInfo: Obtendo informações sobre os bancos de dados Entrez

**pag 154**